# ML with TensorFlow Extended (TFX) -- Part 4
The puprpose of this tutorial is to show how to do end-to-end ML with TFX libraries on Google Cloud Platform. This tutorial covers:
1. Data analysis and schema generation with **TF Data Validation**.
2. Data preprocessing with **TF Transform**.
3. Model training with **TF Estimator**.
4. Model evaluation with **TF Model Analysis**.

This notebook has been tested in Jupyter on the Deep Learning VM.

## Setup Cloud environment

In [ ]:
PROJECT = 'cloud-training-demos'    # Replace with your PROJECT
BUCKET = 'cloud-training-demos-ml'  # Replace with your BUCKET
REGION = 'us-central1'              # Choose an available region for Cloud MLE

import os

os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION

In [ ]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

## ensure we predict locally with our current Python environment
gcloud config set ml_engine/local_python `which python`

<img valign="middle" src="images/tfx.jpeg">

### UCI Adult Dataset: https://archive.ics.uci.edu/ml/datasets/adult
Predict whether income exceeds $50K/yr based on census data. Also known as "Census Income" dataset.

In [ ]:
DATA_DIR='gs://cloud-samples-data/ml-engine/census/data'

In [ ]:
import os

TRAIN_DATA_FILE = os.path.join(DATA_DIR, 'adult.data.csv')
EVAL_DATA_FILE = os.path.join(DATA_DIR, 'adult.test.csv')
!gsutil ls -l $TRAIN_DATA_FILE
!gsutil ls -l $EVAL_DATA_FILE

In [ ]:
HEADER = ['age', 'workclass', 'fnlwgt', 'education', 'education_num',
               'marital_status', 'occupation', 'relationship', 'race', 'gender',
               'capital_gain', 'capital_loss', 'hours_per_week',
               'native_country', 'income_bracket']

TARGET_FEATURE_NAME = 'income_bracket'
TARGET_LABELS = [' <=50K', ' >50K']
WEIGHT_COLUMN_NAME = 'fnlwgt_scaled'

In [ ]:
# from previous step
RAW_SCHEMA_LOCATION = 'raw_schema.pbtxt'
MODEL_DIR = 'models/census/dnn_classifier'
os.environ['MODEL_DIR'] = MODEL_DIR

## 4. Model Evaluation
For assessing the quality of our trained model, we use [TensorFlow Model Analysis](https://www.tensorflow.org/tfx/guide/tfma) to perform the following:
1. **Export** evaluation saved model (we did this in notebook #3 when we trained the model)
2. Define **data slices** for analysis
3. **Visualise** and evaluate the model



In [ ]:
import tensorflow_model_analysis as tfma
import tensorflow as tf

In [ ]:
!ls {MODEL_DIR}/export

### 4.2 Prepare data slices for evaluation

In [ ]:
slice_spec = [
  tfma.slicer.SingleSliceSpec(),
  tfma.slicer.SingleSliceSpec(columns=['occupation'])
]

eval_model_location = os.path.join(eval_model_dir, os.listdir(eval_model_dir)[-1])
print eval_model_location

eval_result = tfma.run_model_analysis(
    eval_shared_model=tfma.default_eval_shared_model(
        eval_saved_model_path=eval_model_location,
        example_weight_key=WEIGHT_COLUMN_NAME) , 
    data_location=EVAL_DATA_FILE, 
    file_format='text', 
    slice_spec=slice_spec,  
    output_path=None
)

In [ ]:
eval_result.slicing_metrics

### 4.3 Visualize and analyze evalation results

In [ ]:
tfma.view.render_slicing_metrics(
    result=eval_result, 
    slicing_column=b'occupation'
)

## License

Copyright 2019 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0.

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

---
This is not an official Google product. The sample code provided for educational purposes only.
---